***
# chapter 8. Analytic Function
***
### keyword
* ORDER BY
* PARTITION BY
* RANK, DENSE_RANK, ROW_NUMBER
* ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW , ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
* FIRST_VALUE, LAST_VALUE

In [1]:
import cx_Oracle
import pandas as pd
xedb = cx_Oracle.connect('hr/hr@localhost/xe')
cur = xedb.cursor()

#### 20번 부서 사원들의 급여 누적 합계

In [2]:
df = pd.read_sql("""
        SELECT e.employee_id, e.salary, e.department_id, SUM(t.salary) AS TOTAL
        FROM employees e, employees t
        WHERE e.employee_id >= t.employee_id
        AND t.department_id = 20
        AND e.department_id = 20
        GROUP BY e.employee_id, e.salary, e.department_id
        ORDER BY e.employee_id
        """,xedb)
print(df)

   EMPLOYEE_ID   SALARY  DEPARTMENT_ID  TOTAL
0          201  13000.0             20  13000
1          202   6000.0             20  19000


=> 문제점 : 너무 많은 스캔을 하고, 테이블 두 개를 access
## < ORDER BY >

In [3]:
df = pd.read_sql("""
        SELECT employee_id, salary, department_id, 
                SUM(salary) OVER (ORDER BY employee_id) AS TOTAL
        FROM employees
        WHERE department_id = 20 
        """,xedb)
print(df)

   EMPLOYEE_ID   SALARY  DEPARTMENT_ID  TOTAL
0          201  13000.0             20  13000
1          202   6000.0             20  19000


In [4]:
# ORDER BY 생략, 총합계산

df = pd.read_sql("""
        SELECT employee_id, salary, department_id, 
            SUM(salary) OVER () AS TOTAL
        FROM employees
        WHERE department_id = 20 
        """,xedb)
print(df)

   EMPLOYEE_ID   SALARY  DEPARTMENT_ID  TOTAL
0          201  13000.0             20  19000
1          202   6000.0             20  19000


## < PARTITION BY >
#### 부서별 급여의 누계, 총액

In [5]:
df = pd.read_sql("""
        SELECT employee_id, salary, department_id,
                sum(salary) OVER (PARTITION BY department_id) AS dept_total,
                sum(salary) OVER (PARTITION BY department_id ORDER BY employee_id) AS runnig_total
        FROM employees
        """,xedb)
print(df)

     EMPLOYEE_ID   SALARY  DEPARTMENT_ID  DEPT_TOTAL  RUNNIG_TOTAL
0            200   4400.0           10.0        4400          4400
1            201  13000.0           20.0       19000         13000
2            202   6000.0           20.0       19000         19000
3            114  11000.0           30.0       24900         11000
4            115   3100.0           30.0       24900         14100
5            116   2900.0           30.0       24900         17000
6            117   2800.0           30.0       24900         19800
7            118   2600.0           30.0       24900         22400
8            119   2500.0           30.0       24900         24900
9            203   6500.0           40.0        6500          6500
10           120   8000.0           50.0      156400          8000
11           121   8200.0           50.0      156400         16200
12           122   7900.0           50.0      156400         24100
13           123   6500.0           50.0      156400         3

#### 사원 전체 평균값

In [6]:
df = pd.read_sql("""
        SELECT employee_id, salary, department_id, round(avg(salary) over(),0) avg_sal
        FROM employees
        """,xedb)
print(df)

     EMPLOYEE_ID   SALARY  DEPARTMENT_ID  AVG_SAL
0            100  24000.0           90.0     6462
1            101  17000.0           90.0     6462
2            102  17000.0           90.0     6462
3            103   9000.0           60.0     6462
4            104   6000.0           60.0     6462
5            105   4800.0           60.0     6462
6            106   4800.0           60.0     6462
7            107   4200.0           60.0     6462
8            108  12008.0          100.0     6462
9            109   9000.0          100.0     6462
10           110   8200.0          100.0     6462
11           111   7700.0          100.0     6462
12           112   7800.0          100.0     6462
13           113   6900.0          100.0     6462
14           114  11000.0           30.0     6462
15           115   3100.0           30.0     6462
16           116   2900.0           30.0     6462
17           117   2800.0           30.0     6462
18           118   2600.0           30.0     6462


## < RANK, DENSE_RANK, ROW_NUMBER >
#### 사원들의 급여 순위

In [7]:
df = pd.read_sql("""
        SELECT employee_id, last_name, salary,
                RANK() OVER (ORDER BY salary DESC) rank,
                DENSE_RANK() OVER (ORDER BY salary DESC) DENSE_RANK,
                ROW_NUMBER () OVER (ORDER BY salary DESC) ROW_NUMBER
         FROM employees
        """,xedb)
print(df)

     EMPLOYEE_ID    LAST_NAME   SALARY  RANK  DENSE_RANK  ROW_NUMBER
0            100         King  24000.0     1           1           1
1            101      Kochhar  17000.0     2           2           2
2            102      De Haan  17000.0     2           2           3
3            145      Russell  14000.0     4           3           4
4            146     Partners  13500.0     5           4           5
5            201    Hartstein  13000.0     6           5           6
6            108    Greenberg  12008.0     7           6           7
7            205      Higgins  12008.0     7           6           8
8            147    Errazuriz  12000.0     9           7           9
9            168         Ozer  11500.0    10           8          10
10           114     Raphaely  11000.0    11           9          11
11           148    Cambrault  11000.0    11           9          12
12           174         Abel  11000.0    11           9          13
13           149      Zlotkey  105

#### 부서별, 부서내 급여순위

In [8]:
df = pd.read_sql("""
        SELECT department_id, salary,
                RANK() OVER (PARTITION BY department_id ORDER BY salary DESC) dept_rank,
                DENSE_RANK() OVER (PARTITION BY department_id ORDER BY salary DESC) dept_dense_rank
        FROM employees
        """,xedb)
print(df)

     DEPARTMENT_ID   SALARY  DEPT_RANK  DEPT_DENSE_RANK
0             10.0   4400.0          1                1
1             20.0  13000.0          1                1
2             20.0   6000.0          2                2
3             30.0  11000.0          1                1
4             30.0   3100.0          2                2
5             30.0   2900.0          3                3
6             30.0   2800.0          4                4
7             30.0   2600.0          5                5
8             30.0   2500.0          6                6
9             40.0   6500.0          1                1
10            50.0   8200.0          1                1
11            50.0   8000.0          2                2
12            50.0   7900.0          3                3
13            50.0   6500.0          4                4
14            50.0   5800.0          5                5
15            50.0   4200.0          6                6
16            50.0   4100.0          7          

## < ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
정렬결과의 처음부터 현재 행 까지를 대상
## ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING >
정렬결과의 처음과 끝 을 대상

In [9]:
df = pd.read_sql("""
        SELECT employee_id, salary,
                sum(salary) OVER (ORDER BY employee_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) running1,
                sum(salary) OVER (ORDER BY employee_id ) running2,
                sum(salary) OVER (ORDER BY employee_id ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) total1,
                sum(salary) OVER () total2  
        FROM employees
        """,xedb)
print(df)

     EMPLOYEE_ID   SALARY  RUNNING1  RUNNING2  TOTAL1  TOTAL2
0            100  24000.0     24000     24000  691416  691416
1            101  17000.0     41000     41000  691416  691416
2            102  17000.0     58000     58000  691416  691416
3            103   9000.0     67000     67000  691416  691416
4            104   6000.0     73000     73000  691416  691416
5            105   4800.0     77800     77800  691416  691416
6            106   4800.0     82600     82600  691416  691416
7            107   4200.0     86800     86800  691416  691416
8            108  12008.0     98808     98808  691416  691416
9            109   9000.0    107808    107808  691416  691416
10           110   8200.0    116008    116008  691416  691416
11           111   7700.0    123708    123708  691416  691416
12           112   7800.0    131508    131508  691416  691416
13           113   6900.0    138408    138408  691416  691416
14           114  11000.0    149408    149408  691416  691416
15      

## < FIRST_VALUE, LAST_VALUE >
#### 부서별 최소급여, 최대급여 

In [10]:
df = pd.read_sql("""
        SELECT department_id,salary,
                first_value(salary) OVER (PARTITION BY department_id ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) first_value,
                last_value(salary) OVER (PARTITION BY department_id ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) last_value
        FROM employees
        """,xedb)
print(df)

     DEPARTMENT_ID   SALARY  FIRST_VALUE  LAST_VALUE
0             10.0   4400.0         4400        4400
1             20.0   6000.0         6000       13000
2             20.0  13000.0         6000       13000
3             30.0   2500.0         2500       11000
4             30.0   2600.0         2500       11000
5             30.0   2800.0         2500       11000
6             30.0   2900.0         2500       11000
7             30.0   3100.0         2500       11000
8             30.0  11000.0         2500       11000
9             40.0   6500.0         6500        6500
10            50.0   2100.0         2100        8200
11            50.0   2200.0         2100        8200
12            50.0   2200.0         2100        8200
13            50.0   2400.0         2100        8200
14            50.0   2400.0         2100        8200
15            50.0   2500.0         2100        8200
16            50.0   2500.0         2100        8200
17            50.0   2500.0         2100      

# [ 연습문제 ]
### (1) 자신의 부서 평균 급여보다 급여를 많이 받은 사원들의 employee_id, salary, department_name를 조회하세요
####  < SUBQUERY >

In [11]:
df = pd.read_sql("""
        SELECT e.employee_id, e.salary, d.department_name
        FROM employees e, departments d
        WHERE e.department_id = d.department_id
        AND e.salary > (SELECT avg(salary)
                        FROM employees 
                        WHERE department_id = d.department_id)
        """,xedb)
print(df)  

    EMPLOYEE_ID   SALARY DEPARTMENT_NAME
0           201  13000.0       Marketing
1           114  11000.0      Purchasing
2           120   8000.0        Shipping
3           121   8200.0        Shipping
4           122   7900.0        Shipping
5           123   6500.0        Shipping
6           124   5800.0        Shipping
7           137   3600.0        Shipping
8           141   3500.0        Shipping
9           184   4200.0        Shipping
10          185   4100.0        Shipping
11          188   3800.0        Shipping
12          189   3600.0        Shipping
13          192   4000.0        Shipping
14          193   3900.0        Shipping
15          103   9000.0              IT
16          104   6000.0              IT
17          145  14000.0           Sales
18          146  13500.0           Sales
19          147  12000.0           Sales
20          148  11000.0           Sales
21          149  10500.0           Sales
22          150  10000.0           Sales
23          151 

#### < INLINE VIEW >

In [12]:
df = pd.read_sql("""
        SELECT e.employee_id, e.salary, d.department_name
        FROM (SELECT department_id, avg(salary) avg_sal
              FROM employees
              GROUP BY department_id) a, employees e, departments d
        WHERE e.department_id = d.department_id
        AND a.department_id = e.department_id
        AND e.salary > A.avg_sal
        """,xedb)
print(df)

    EMPLOYEE_ID   SALARY DEPARTMENT_NAME
0           109   9000.0         Finance
1           108  12008.0         Finance
2           114  11000.0      Purchasing
3           100  24000.0       Executive
4           201  13000.0       Marketing
5           205  12008.0      Accounting
6           193   3900.0        Shipping
7           192   4000.0        Shipping
8           189   3600.0        Shipping
9           188   3800.0        Shipping
10          185   4100.0        Shipping
11          184   4200.0        Shipping
12          141   3500.0        Shipping
13          137   3600.0        Shipping
14          124   5800.0        Shipping
15          123   6500.0        Shipping
16          122   7900.0        Shipping
17          121   8200.0        Shipping
18          120   8000.0        Shipping
19          174  11000.0           Sales
20          170   9600.0           Sales
21          169  10000.0           Sales
22          168  11500.0           Sales
23          163 

#### < 분석함수 사용 >

In [13]:
df = pd.read_sql("""
        SELECT employee_id, salary, department_name
        FROM (SELECT e.employee_id, e.salary, d.department_name,
              CASE WHEN e.salary > avg(salary) OVER (PARTITION BY e.department_id) THEN '1' END col_1
              FROM employees e, departments d
              WHERE e.department_id = d.department_id)
        WHERE col_1 = 1
        """,xedb)
print(df)

# WHERE col_1 IS NOT NULL
# IF TRUE, 1 /IF FALSE, null

    EMPLOYEE_ID   SALARY DEPARTMENT_NAME
0           201  13000.0       Marketing
1           114  11000.0      Purchasing
2           120   8000.0        Shipping
3           121   8200.0        Shipping
4           122   7900.0        Shipping
5           123   6500.0        Shipping
6           124   5800.0        Shipping
7           137   3600.0        Shipping
8           141   3500.0        Shipping
9           184   4200.0        Shipping
10          185   4100.0        Shipping
11          188   3800.0        Shipping
12          189   3600.0        Shipping
13          192   4000.0        Shipping
14          193   3900.0        Shipping
15          103   9000.0              IT
16          104   6000.0              IT
17          145  14000.0           Sales
18          146  13500.0           Sales
19          147  12000.0           Sales
20          148  11000.0           Sales
21          149  10500.0           Sales
22          150  10000.0           Sales
23          151 

=> 서브쿼리로 수행해도 분석함수로 작성했던 실행계획으로 트랜스포밍된다.

### (2) 사원들의 급여순위에서 2등까지 조회하세요
#### < ROW_NUMBER >
급여 중에 중복이 없을 때만 사용할 수 있다. 중복이 있을 경우 누락된다.

In [14]:
df = pd.read_sql("""
        SELECT rownum, last_name, salary
        FROM (SELECT last_name, salary 
              FROM employees ORDER BY salary DESC)
        WHERE ROWNUM <= 2
        """,xedb)
print(df)

   ROWNUM LAST_NAME   SALARY
0       1      King  24000.0
1       2   Kochhar  17000.0


#### < DENSE_RANK >

In [15]:
df = pd.read_sql("""
        SELECT  rank, last_name, salary
        FROM (SELECT DENSE_RANK() OVER(ORDER BY salary DESC) as rank, last_name, salary
              FROM employees)
        WHERE rank <= 2
        """,xedb)
print(df)

   RANK LAST_NAME   SALARY
0     1      King  24000.0
1     2   Kochhar  17000.0
2     2   De Haan  17000.0


In [16]:
cur.close()
xedb.close()